In [1]:
import firebase_admin
from firebase_admin import credentials, db

# ================= 🧹 KONFIGURASI =================
# Ganti dengan path file JSON Anda
CRED_PATH = "/work/staklimjerukagung-firebase-admin.json" 
DB_URL = "https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/"
# Path di database tempat data sensor disimpan
# Misal: sensors/kbm-05/logs
DB_NODE_PATH = "/auto_weather_stat/id-05/data"

# ==================================================

# Setup
if not firebase_admin._apps:
    cred = credentials.Certificate(CRED_PATH)
    firebase_admin.initialize_app(cred, {'databaseURL': DB_URL})

def find_and_destroy_zeros():
    ref = db.reference(DB_NODE_PATH)
    keys_to_delete = set() # Pakai 'set' biar tidak ada key ganda
    
    print("🔍 Sedang memindai database untuk nilai 0...")

    # --- LANGKAH 1: Cari Temperature = 0 ---
    print("   👉 Mengecek Temperature == 0...")
    # Query: Order by 'temperature', Equal to 0
    snapshot_temp = ref.order_by_child('temperature').equal_to(0).get()
    
    if snapshot_temp:
        for key, val in snapshot_temp.items():
            print(f"      Found (Temp=0): {key} -> {val}")
            keys_to_delete.add(key)
    
    # --- LANGKAH 2: Cari Humidity = 0 ---
    print("   👉 Mengecek Humidity == 0...")
    # Query: Order by 'humidity', Equal to 0
    snapshot_hum = ref.order_by_child('humidity').equal_to(0).get()
    
    if snapshot_hum:
        for key, val in snapshot_hum.items():
            print(f"      Found (Hum=0): {key} -> {val}")
            keys_to_delete.add(key)

    # --- LANGKAH 3: Eksekusi Hapus ---
    total = len(keys_to_delete)
    
    if total == 0:
        print("\n✅ Aman! Tidak ditemukan data dengan nilai 0.")
        return

    print(f"\n🚨 DITEMUKAN {total} DATA SAMPAH (Nilai 0).")
    print("Data ini akan DIHAPUS PERMANEN dari database.")
    
    confirm = input("Ketik 'YAKIN' untuk menghapus: ")
    
    if confirm == 'YAKIN':
        print("\n🔥 Menghapus...")
        count = 0
        for key in keys_to_delete:
            try:
                ref.child(key).delete()
                print(f"   🗑️ Deleted: {key}")
                count += 1
            except Exception as e:
                print(f"   ❌ Gagal hapus {key}: {e}")
        print(f"✨ Selesai! {count} data berhasil dihapus.")
    else:
        print("Dibatalkan.")

if __name__ == "__main__":
    find_and_destroy_zeros()

🔍 Sedang memindai database untuk nilai 0...
   👉 Mengecek Temperature == 0...


InvalidArgumentError: Index not defined, add ".indexOn": "temperature", for path "/auto_weather_stat/id-05/data", to the rules

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>